In [ ]:
# !pip install -U pymilvus

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from pymilvus import MilvusClient

milvus_client = MilvusClient("./milvus_demo.db")

2025-02-07 23:16:14,460 [ERROR][_create_connection]: Failed to create new connection using: efc99d0be6714e17a061f69514454b86 (milvus_client.py:920)


ModuleNotFoundError: No module named 'milvus_lite'

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
# Milvus Lite（ミルバスライト）のライブラリ
from milvus import MilvusLite, Collection, CollectionSchema, FieldSchema, DataType

# --- Milvus Lite（ミルバスライト）クライアントの作成 ---
# Milvus Lite はローカルで動作するため、サーバ接続は不要です
client = MilvusLite(in_memory=True)  # メモリ内で利用する場合（永続化する場合はファイルパスを指定）

# --- コレクションの作成 ---
# DreamSim（ドリームシム）の出力次元に合わせます。ここでは例として768次元を使用
dim = 768
collection_name = "image_collection"

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim)
]
schema = CollectionSchema(fields, description="Image embeddings collection")

# 既存のコレクションがあれば削除（Milvus Lite の API に合わせて実装）
if collection_name in client.list_collections():
    client.drop_collection(collection_name)

collection = client.create_collection(name=collection_name, schema=schema)

# --- DreamSim（ドリームシム）によるベクトル化 ---
# ※ 以下は DreamSim モデルと前処理関数のサンプルです。実際には環境に合わせて定義してください。
# model = ...       # DreamSim のモデルインスタンス
# preprocess = ...  # 画像の前処理関数

def get_normalized_embedding(image):
    """
    画像を DreamSim（ドリームシム）でベクトル化し、L2正規化を行う関数。
    モデル出力が既に正規化されている場合は、この処理は不要です。
    """
    with torch.no_grad():
        # GPUへ転送してベクトル化（出力は形状 (1, dim) を想定）
        emb = model.embed(preprocess(image).to("cuda")).cpu().numpy().squeeze()
    norm = np.linalg.norm(emb)
    return emb if norm == 0 else emb / norm

# --- 画像データのインデックス作成 ---
# images: 検索対象の画像リスト（例）
# images = [...]

# 各画像の埋め込みを取得し、Milvus Lite（ミルバスライト）に登録
embeddings = [get_normalized_embedding(im) for im in images]
entities = {"embedding": embeddings}
collection.insert(entities)
collection.flush()

# --- クエリ画像による検索 ---
# query: クエリ画像（例）
# query = ...

query_emb = get_normalized_embedding(query)

# Milvus Lite では、正規化済みベクトルの場合、内積（IP）を用いることでコサイン類似度と同等の検索が可能です
search_params = {"metric_type": "IP", "params": {"nprobe": 10}}

results = collection.search(
    [query_emb],
    "embedding",
    search_params,
    limit=10,
    output_fields=["id"]
)

# 検索結果の整形（内積は正規化済みならコサイン類似度と同じ）
result_list = []
for hits in results:
    for hit in hits:
        cosine_sim = hit.distance  # 内積値（正規化済みならばコサイン類似度）
        cosine_distance = 1 - cosine_sim
        result_list.append({"id": hit.id, "cosine_distance": cosine_distance})

df = pd.DataFrame(result_list).sort_values(by="cosine_distance")
print(df)
